# **Subqueries**

A **subquery** is a query **nested** inside of another query, the advantage of doing this is that it allows
- *Comparing* grouped data to summarized values
- *Reshaping* of data
- *Combining* of data where a join or set operation is not possible

In [1]:
%load_ext sql
%sql sqlite:///databases/world.db

## Subqueries Inside `SELECT`

If a query returns a single column, then that column can be used in a new table.

In [19]:
%%sql
SELECT 
    DISTINCT continent, 
    (
        SELECT COUNT(*)
        FROM monarchs
        WHERE states.continent = monarchs.continent
    ) AS monarch_count
FROM states
ORDER BY monarch_count DESC
LIMIT(5)

 * sqlite:///databases/world.db
Done.


continent,monarch_count
Asia,2
Europe,2
Oceania,0
South America,0
Africa,0


Things to keep in mind
- Return a single value (to be compared with each row value of the main table)
- Preferably apply the same filter to  the main and sub query 


In [20]:
%sql sqlite:///databases/football.db

In [26]:
%%sql 
SELECT 
    date,
    (home_goal + away_goal) AS goals,
    ROUND((home_goal + away_goal) - (
        SELECT AVG(home_goal+away_goal) 
        FROM matches
        WHERE season = "2011/2012"
    ), 2) AS diff
FROM matches
WHERE season = "2011/2012"
LIMIT(5)

 * sqlite:///databases/football.db
   sqlite:///databases/world.db
Done.


date,goals,diff
2011-07-29T00:00:00.000,3,0.28
2011-07-30T00:00:00.000,2,-0.72
2011-07-30T00:00:00.000,4,1.28
2011-07-30T00:00:00.000,1,-1.72
2011-07-30T00:00:00.000,0,-2.72
